# Finance Headline QA Dataset Preparation

本笔试任务旨在将 AdaptLLM/finance-tasks 数据集中 Headline 子集的文本数据转换成结构化问答对 JSON 格式。

- 自动下载数据文件
- 解析文本中的多组问答
- 输出格式化 JSON

---

代码步骤清晰，运行时间有统计，方便查看效果。


In [1]:
!pip install datasets --quiet

In [6]:
import requests
import json
import uuid
import re
import time

def extract_qa_pairs(text):
    qa_list = []
    questions = re.findall(r'([^\?]*\?)', text)
    pos = 0
    for q in questions:
        start = text.find(q, pos) + len(q)
        snippet = text[start:start+30]
        match = re.search(r'\b(Yes|No|yes|no|-?\d+\.?\d*)\b', snippet)
        answer = match.group(1) if match else "Unknown"
        qa_list.append({
            "id": str(uuid.uuid4()),
            "Question": q.strip(),
            "Answer": answer,
            "Tag": "finance-headline"
        })
        pos = start
    return qa_list

def main():
    start_time = time.time()

    # 1. 用requests下载文件
    url = "https://huggingface.co/datasets/AdaptLLM/finance-tasks/resolve/main/Headline/test.json"
    print("开始下载文件...")
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    print(f"文件下载成功，包含条目数量: {len(data)}")

    # 2. 遍历数据，抽取问答对
    all_qa_pairs = []
    for item in data:
        # 这里假设问答文本在item的某字段，例如"input"，根据实际结构调整
        text = item.get("input", "")
        qa_pairs = extract_qa_pairs(text)
        all_qa_pairs.extend(qa_pairs)

    # 3. 保存结果
    with open("finance_headline_qa.json", "w", encoding="utf-8") as f:
        json.dump(all_qa_pairs, f, ensure_ascii=False, indent=2)

    end_time = time.time()
    print(f"共提取问答对数量: {len(all_qa_pairs)}")
    print(f"数据处理总耗时（秒）: {end_time - start_time:.2f}")

if __name__ == "__main__":
    main()


开始下载文件...
文件下载成功，包含条目数量: 20547
共提取问答对数量: 146020
数据处理总耗时（秒）: 3.38
